In [1]:
# check installation of python 
! which python

# check installation of pip
! which pip

# upgrade pip if needed 
! pip install --upgrade pip

# install biopython
! pip install biopython

# upgrade biopython if needed 

! pip install biopython --upgrade

# import biopython and check import
import Bio
print(Bio.__version__)

/Users/giana.cirolia/anaconda3/bin/python
/Users/giana.cirolia/anaconda3/bin/pip
     |████████████████████████████████| 1.5 MB 632 kB/s eta 0:00:0104
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3
  Using cached biopython-1.78-cp36-cp36m-macosx_10_9_x86_64.whl (2.2 MB)
     |████████████████████████████████| 2.3 MB 1.9 MB/s eta 0:00:01     |█████████████▉                  | 972 kB 269 kB/s eta 0:00:05
1.78


In [5]:
## import command line interface for biopython blasts 
## more documentation here https://biopython.org/docs/1.75/api/Bio.Blast.Applications.html
from Bio.Blast.Applications import NcbiblastnCommandline

In [95]:
! mkdir -p database/output
cline = NcbiblastnCommandline(query="../sequences/queries/D.mel_multiple_seqs.fasta", db="../database/genomes_db", strand="both",
                              evalue=100, out="../database/output/D.mel_blast.xml", outfmt=5, task= "dc-megablast")
cline
print(cline)
cline()

blastn -out ../database/output/D.mel_blast.xml -outfmt 5 -query ../sequences/queries/D.mel_multiple_seqs.fasta -db ../database/genomes_db -evalue 100 -strand both -task dc-megablast


('', '')

In [113]:
## parsingg XML blast output for one test record 
## https://biopython.org/docs/1.75/api/Bio.Blast.Record.html
## https://biopython.org/docs/1.75/api/Bio.Blast.NCBIXML.html

from Bio.Blast import NCBIXML

record_handle = open("../database/output/D.mel_blast.xml", 'r')

blast_records = NCBIXML.parse(record_handle)
blast_record = next(blast_records)

E_VALUE_THRESH = 1e-7 # this is the number of random hits expected at the same blast score level as your hit
MATCH_LENGTH = 40 # this is the number of residues you would like to match to local but non continious region

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        contig_length= alignment.length
        contig_name = alignment.title
        query_match_start = hsp.query_start
        query_match_end = hsp.query_end
        contig_match_start = hsp.sbjct_start
        contig_match_end = hsp.sbjct_end
        total_match = query_match_end - query_match_start
        e_value = hsp.expect
        BLAST_score = hsp.score      
        if hsp.expect< E_VALUE_THRESH and total_match > MATCH_LENGTH:
            print("******Alignment******")
            print("sequence:", contig_name) ## name of contig in file 
            print("length of contig: {}".format(contig_length)) ## length of whole contig
            print("query_match_start_residue: {}, query_match_end_residue: {}".format(query_match_start, query_match_end))
            print("sequence match start: {}, sequence match end: {}".format(contig_match_start , contig_match_end))
            print("e value:", e_value)
            print("BLAST score:", BLAST_score)
            print(hsp.query[0:100]) ## search seq
            print(hsp.match[0:100]) ## lines connecting them
            print(hsp.sbjct[0:100]) ## contig matched to 
            




******Alignment******
sequence: contig_1--organism-Drosophila-melanogaster No definition line
length of contig: 21632781
query_match_start_residue: 1, query_match_end_residue: 798
sequence match start: 6255858, sequence match end: 6256655
e value: 0.0
BLAST score: 1596.0
AATATAACCCAATAATTTGAAGTAACTGGCAGGAGCGAGGTATCCTTCCTGGTTACCCGGTACTGCATAACAATGGAACCCGAACCGTAACTGGGACAGA
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
AATATAACCCAATAATTTGAAGTAACTGGCAGGAGCGAGGTATCCTTCCTGGTTACCCGGTACTGCATAACAATGGAACCCGAACCGTAACTGGGACAGA
******Alignment******
sequence: contig_529--organism-Drosophila-sechellia No definition line
length of contig: 19765707
query_match_start_residue: 1, query_match_end_residue: 798
sequence match start: 15052703, sequence match end: 15051916
e value: 0.0
BLAST score: 1421.0
AATATAACCCAATAATTTGAAGTAACTGGCAGGAGCGAGGTATCCTTCCTGGTTACCCGGTACTGCATAACAATGGAACCCGAACCGTAACTGGGACAGA
||||||||||||||||||||||||||| |||||||||||||||||||| ||

In [112]:
E_VALUE_THRESH = 1e-7
for record in NCBIXML.parse(open("../database/output/D.mel_blast.xml", 'r')): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
           for hsp in align.hsps: 
              if hsp.expect < E_VALUE_THRESH: 
                 print("match: %s " % align.title[:100])



query: dmel_stripe2_full_jake
match: contig_1--organism-Drosophila-melanogaster No definition line 
match: contig_529--organism-Drosophila-sechellia No definition line 
match: contig_203--organism-Drosophila-simulans No definition line 
match: contig_22--organism-Drosophila-mauritiana No definition line 
match: contig_380--organism-Chymomyza-costata No definition line 


query: dmel_stripe2_minimal_bronsky
match: contig_1--organism-Drosophila-melanogaster No definition line 
match: contig_529--organism-Drosophila-sechellia No definition line 
match: contig_203--organism-Drosophila-simulans No definition line 
match: contig_22--organism-Drosophila-mauritiana No definition line 
match: contig_380--organism-Chymomyza-costata No definition line 


query: dmel_stripe2_gene_NCBI
match: contig_1--organism-Drosophila-melanogaster No definition line 
match: contig_529--organism-Drosophila-sechellia No definition line 
match: contig_22--organism-Drosophila-mauritiana No definition line 
match:

In [94]:
from collections import defaultdict 

E_VALUE_THRESH = .01

results = defaultdict(list)

for record in NCBIXML.parse(open("../database/output/D.mel_blast.xml", 'r')): 
     if record.descriptions: 
        for desc in record.descriptions:
            if desc.e < E_VALUE_THRESH:
                results[record.query].append(desc.title)

if list(results["dmel_stripe2_full_jake"]).sort() == list(results["dmel_stripe2_minimal_bronsky"]).sort():
    print("lists overlap entirely")

                    

lists overlap entirely
